## The notebook implementation of Research paper summarizer:

---

In [ ]:
!pip install transformers datasets torch

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name,clean_up_tokenization_spaces= True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("scientific_papers", "arxiv")

README.md:   0%|          | 0.00/8.27k [00:00<?, ?B/s]

scientific_papers.py:   0%|          | 0.00/5.35k [00:00<?, ?B/s]

The repository for scientific_papers contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scientific_papers.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/203037 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6436 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 203037
    })
    validation: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 6436
    })
    test: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 6440
    })
})


In [4]:
def preprocess_func(text):
    inputs = text['article']
    targets= text['abstract']
    model_inputs = tokenizer(inputs, max_length = 1024, truncation= True, padding= 'max_length')
    labels = tokenizer(targets, max_length= 256, truncation= True, padding= 'max_length')
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset= dataset.map(preprocess_func, batched= True)

Map:   0%|          | 0/203037 [00:00<?, ? examples/s]

Map:   0%|          | 0/6436 [00:00<?, ? examples/s]

Map:   0%|          | 0/6440 [00:00<?, ? examples/s]

In [13]:
# #@ Saving the mapped dataset so that it becomes effecient if training error occours 

# tokenized_dataset.save_to_disk("mapped_dataset_dir")

Saving the dataset (0/18 shards):   0%|          | 0/203037 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6436 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6440 [00:00<?, ? examples/s]

In [ ]:
# !zip -r compressed_mapped.zip mapped_dataset_dir

Okay, now I'm not mapping it again. Instead, I'll unzip the mapped_value and run from this. 

In [5]:
from datasets import load_from_disk

mapped_df= load_from_disk('/kaggle/input/compressed-mapped-files/mapped_dataset_dir')

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

In [6]:
from transformers import Trainer, TrainingArguments

training_args= TrainingArguments(
    output_dir= "./results",
    eval_strategy= "epoch", 
    save_strategy="epoch",
    save_total_limit=3,
    learning_rate= 2e-5, 
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs= 3, 
    weight_decay = 0.01, 
    logging_dir="./logs",
    fp16 = True
)

In [8]:
# small_train = tokenized_dataset["train"].select(range(10000))
# small_eval = tokenized_dataset["validation"].select(range(4000))

In [7]:
small_train = mapped_df["train"].select(range(10000))
small_eval = mapped_df["validation"].select(range(2000))

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_eval,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [9]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api_key")

In [10]:
import wandb
wandb.login(key= secret_value_0)
wandb.init(project="Researchpaper_summarize", name="wtf")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: firojpaudel (firojpaudel-madan-bhandari-memorial-college). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [11]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,2.141400,1.818125
2,1.867500,1.777878
3,1.785800,1.769484


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-mod

TrainOutput(global_step=1875, training_loss=1.8924777669270834, metrics={'train_runtime': 10015.5745, 'train_samples_per_second': 2.995, 'train_steps_per_second': 0.187, 'total_flos': 6.501313806336e+16, 'train_loss': 1.8924777669270834, 'epoch': 3.0})

In [13]:
import shutil
import os
logs_dir = '/kaggle/working/ft_summ'
try:
    if os.path.exists(logs_dir):
        shutil.rmtree(logs_dir)  # Remove the directory and its contents
        print(f"Directory '{logs_dir}' removed successfully.")
    else:
        print(f"Directory '{logs_dir}' does not exist.")
except OSError as e:
    print(f"Error removing directory '{logs_dir}': {e}")

Directory '/kaggle/working/ft_summ' removed successfully.


In [14]:
##@ saving the finetuned model:
trainer.save_model("./fine_tuned_bart")
tokenizer.save_pretrained("./fine_tuned_bart")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./fine_tuned_bart/tokenizer_config.json',
 './fine_tuned_bart/special_tokens_map.json',
 './fine_tuned_bart/vocab.json',
 './fine_tuned_bart/merges.txt',
 './fine_tuned_bart/added_tokens.json',
 './fine_tuned_bart/tokenizer.json')

In [17]:
import os

print(os.listdir("./fine_tuned_bart"))

['model.safetensors', 'training_args.bin', 'special_tokens_map.json', 'vocab.json', 'tokenizer_config.json', 'tokenizer.json', 'config.json', 'generation_config.json', 'merges.txt']


all the weights are there in model.safetensors and training arguments in training_args.bins

In [19]:
#@ Testing once before downloading the model 

model= AutoModelForSeq2SeqLM.from_pretrained('/kaggle/working/fine_tuned_bart')
tokenizer= AutoTokenizer.from_pretrained('/kaggle/working/fine_tuned_bart')

In [20]:
def summarize(testing):
    #@ Tokenizng the inp text to test in the finetuned model
    inputs = tokenizer(testing, max_length= 1024, truncation= True, return_tensors='pt')

    #@ Generating the summary
    summary_enc = model.generate(inputs['input_ids'], max_length= 500, min_length= 250, length_penalty= 2.0, num_beams=4, early_stopping= True)

    #@ Decoding the generated summary
    summary_fin = tokenizer.decode(summary_enc[0], skip_special_tokens= True)

    return summary_fin

In [21]:
input = '''
BERT: Pre-training of Deep Bidirectional Transformers for
 Language Understanding
 Jacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova
 Google AI Language
 jacobdevlin,mingweichang,kentonl,kristout@google.com
 Abstract
 arXiv:1810.04805v2  [cs.CL]  24 May 2019
 We introduce a new language representa
tion model called BERT, which stands for
 Bidirectional Encoder Representations from
 Transformers. Unlike recent language repre
sentation models (Peters et al., 2018a; Rad
ford et al., 2018), BERT is designed to pre
train deep bidirectional representations from
 unlabeled text by jointly conditioning on both
 left and right context in all layers. As a re
sult, the pre-trained BERT model can be fine
tuned with just one additional output layer
 to create state-of-the-art models for a wide
 range of tasks, such as question answering and
 language inference, without substantial task
specific architecture modifications.
 BERT is conceptually simple and empirically
 powerful. It obtains new state-of-the-art re
sults on eleven natural language processing
 tasks, including pushing the GLUE score to
 80.5% (7.7% point absolute improvement),
 MultiNLI accuracy to 86.7% (4.6% absolute
 improvement), SQuAD v1.1 question answer
ing Test F1 to 93.2 (1.5 point absolute im
provement) and SQuAD v2.0 Test F1 to 83.1
 (5.1 point absolute improvement).
 1 Introduction
 Language model pre-training has been shown to
 be effective for improving many natural language
 processing tasks (Dai and Le, 2015; Peters et al.,
 2018a; Radford et al., 2018; Howard and Ruder,
 2018). These include sentence-level tasks such as
 natural language inference (Bowman et al., 2015;
 Williams et al., 2018) and paraphrasing (Dolan
 and Brockett, 2005), which aim to predict the re
lationships between sentences by analyzing them
 holistically, as well as token-level tasks such as
 named entity recognition and question answering,
 where models are required to produce fine-grained
 output at the token level (Tjong Kim Sang and
 De Meulder, 2003; Rajpurkar et al., 2016).
 There are two existing strategies for apply
ing pre-trained language representations to down
stream tasks: feature-based and fine-tuning. The
 feature-based approach, such as ELMo (Peters
 et al., 2018a), uses task-specific architectures that
 include the pre-trained representations as addi
tional features. The fine-tuning approach, such as
 the Generative Pre-trained Transformer (OpenAI
 GPT) (Radford et al., 2018), introduces minimal
 task-specific parameters, and is trained on the
 downstream tasks by simply fine-tuning all pre
trained parameters. The two approaches share the
 sameobjective function during pre-training, where
 they use unidirectional language models to learn
 general language representations.
 We argue that current techniques restrict the
 power of the pre-trained representations, espe
cially for the fine-tuning approaches. The ma
jor limitation is that standard language models are
 unidirectional, and this limits the choice of archi
tectures that can be used during pre-training. For
 example, in OpenAIGPT,theauthors use aleft-to
right architecture, where every token can only at
tend to previous tokens in the self-attention layers
 of the Transformer (Vaswani et al., 2017). Such re
strictions are sub-optimal for sentence-level tasks,
 and could be very harmful when applying fine
tuning based approaches to token-level tasks such
 as question answering, where it is crucial to incor
porate context from both directions.
 In this paper, we improve the fine-tuning based
 approaches by proposing BERT: Bidirectional
 Encoder Representations from Transformers.
 BERT alleviates the previously mentioned unidi
rectionality constraint by using a “masked lan
guage model” (MLM) pre-training objective, in
spired by the Cloze task (Taylor, 1953). The
 masked language model randomly masks some of
 the tokens from the input, and the objective is to
 predict the original vocabulary id of the masked
word based only on its context. Unlike left-to
right language model pre-training, the MLM ob
jective enables the representation to fuse the left
 and the right context, which allows us to pre
train a deep bidirectional Transformer. In addi
tion to the masked language model, we also use
 a “next sentence prediction” task that jointly pre
trains text-pair representations. The contributions
 of our paper are as follows:
 • Wedemonstrate the importance of bidirectional
 pre-training for language representations. Un
like Radford et al. (2018), which uses unidirec
tional language models for pre-training, BERT
 uses masked language models to enable pre
trained deep bidirectional representations. This
 is also in contrast to Peters et al. (2018a), which
 uses a shallow concatenation of independently
 trained left-to-right and right-to-left LMs.
 • Weshowthatpre-trained representations reduce
 the need for many heavily-engineered task
specific architectures. BERT is the first fine
tuning based representation model that achieves
 state-of-the-art performance on a large suite
 of sentence-level and token-level tasks, outper
forming many task-specific architectures.
 • BERT advances the state of the art for eleven
 NLP tasks. The code and pre-trained mod
els are available at https://github.com/
 google-research/bert.
 2 Related Work
 There is a long history of pre-training general lan
guage representations, and we briefly review the
 most widely-used approaches in this section.
 2.1 Unsupervised Feature-based Approaches
 Learning widely applicable representations of
 words has been an active area of research for
 decades, including non-neural (Brown et al., 1992;
 Ando and Zhang, 2005; Blitzer et al., 2006) and
 neural (Mikolov et al., 2013; Pennington et al.,
 2014) methods. Pre-trained word embeddings
 are an integral part of modern NLP systems, of
fering significant improvements over embeddings
 learned from scratch (Turian et al., 2010). To pre
train word embedding vectors, left-to-right lan
guage modeling objectives have been used (Mnih
 and Hinton, 2009), as well as objectives to dis
criminate correct from incorrect words in left and
 right context (Mikolov et al., 2013).
 These approaches have been generalized to
 coarser granularities, such as sentence embed
dings (Kiros et al., 2015; Logeswaran and Lee,
 2018) or paragraph embeddings (Le and Mikolov,
 2014). To train sentence representations, prior
 work has used objectives to rank candidate next
 sentences (Jernite et al., 2017; Logeswaran and
 Lee, 2018), left-to-right generation of next sen
tence words given a representation of the previous
 sentence (Kiros et al., 2015), or denoising auto
encoder derived objectives (Hill et al., 2016).
 ELMo and its predecessor (Peters et al., 2017,
 2018a) generalize traditional word embedding re
search along a different dimension. They extract
 context-sensitive features from a left-to-right and a
 right-to-left language model. The contextual rep
resentation of each token is the concatenation of
 the left-to-right and right-to-left representations.
 When integrating contextual word embeddings
 with existing task-specific architectures, ELMo
 advances the state of the art for several major NLP
 benchmarks (Peters et al., 2018a) including ques
tion answering (Rajpurkar et al., 2016), sentiment
 analysis (Socher et al., 2013), and named entity
 recognition (Tjong Kim Sang and De Meulder,
 2003). Melamud et al. (2016) proposed learning
 contextual representations through a task to pre
dict a single word from both left and right context
 using LSTMs. Similar to ELMo, their model is
 feature-based and not deeply bidirectional. Fedus
 et al. (2018) shows that the cloze task can be used
 to improve the robustness of text generation mod
els.
 2.2 Unsupervised Fine-tuning Approaches
 As with the feature-based approaches, the first
 works in this direction only pre-trained word em
bedding parameters from unlabeled text (Col
lobert and Weston, 2008).
 More recently, sentence or document encoders
 which produce contextual token representations
 have been pre-trained from unlabeled text and
 f
 ine-tuned for a supervised downstream task (Dai
 and Le, 2015; Howard and Ruder, 2018; Radford
 et al., 2018). The advantage of these approaches
 is that few parameters need to be learned from
 scratch. At least partly due to this advantage,
 OpenAI GPT (Radford et al., 2018) achieved pre
viously state-of-the-art results on many sentence
level tasks from the GLUE benchmark (Wang
 et al., 2018a). Left-to-right language model
NSP
 C
 Mask LM
 ...
 T1
 Mask LM
 T[SEP]
 TN
 T1
 ’ ...
 TM
 ’
 BERT
 MNLI
 NER
 SQuAD
 C
 ...
 T1
 Start/End Span
 T[SEP]
 TN
 T1
 ’ ...
 TM
 ’
 BERT
 BERT
 E[CLS]
 ...
 E1
 E[SEP]
 EN
 E1
 ’ ... EM
 ’
 [CLS]
 Tok 1
 Tok N
 ...
 [SEP]
 Masked Sentence A
 Masked Sentence B
 Unlabeled Sentence A and B Pair 
E[CLS]
 ...
 E1
 E[SEP]
 EN
 E1
 ’ ... EM
 ’
 Tok 1
 TokM
 ...
 [CLS]
 Tok 1
 Tok N
 ...
 Question
 [SEP]
 Tok 1
 TokM
 ...
 Paragraph
 Question Answer Pair
 Pre-training
 Fine-Tuning
 Figure 1: Overall pre-training and fine-tuning procedures for BERT. Apart from output layers, the same architec
tures are used in both pre-training and fine-tuning. The same pre-trained model parameters are used to initialize
 models for different down-stream tasks. During fine-tuning, all parameters are fine-tuned. [CLS] is a special
 symbol added in front of every input example, and [SEP] is a special separator token (e.g. separating ques
tions/answers).
 ing and auto-encoder objectives have been used
 for pre-training such models (Howard and Ruder,
 2018; Radford et al., 2018; Dai and Le, 2015).
 2.3 Transfer Learning from Supervised Data
 There has also been work showing effective trans
fer from supervised tasks with large datasets, such
 as natural language inference (Conneau et al.,
 2017) and machine translation (McCann et al.,
 2017). Computer vision research has also demon
strated the importance of transfer learning from
 large pre-trained models, where an effective recipe
 is to fine-tune models pre-trained with Ima
geNet (Deng et al., 2009; Yosinski et al., 2014).
 3 BERT
 We introduce BERT and its detailed implementa
tion in this section. There are two steps in our
 framework: pre-training and fine-tuning. Dur
ing pre-training, the model is trained on unlabeled
 data over different pre-training tasks. For fine
tuning, the BERT model is first initialized with
 the pre-trained parameters, and all of the param
eters are fine-tuned using labeled data from the
 downstream tasks. Each downstream task has sep
arate fine-tuned models, even though they are ini
tialized with the same pre-trained parameters. The
 question-answering example in Figure 1 will serve
 as a running example for this section.
 A distinctive feature of BERT is its unified ar
chitecture across different tasks. There is mini
mal difference between the pre-trained architec
ture and the final downstream architecture.
 Model Architecture BERT’s model architec
ture is a multi-layer bidirectional Transformer en
coder based on the original implementation de
scribed in Vaswani et al. (2017) and released in
 the tensor2tensor library.1 Because the use
 of Transformers has become common and our im
plementation is almost identical to the original,
 we will omit an exhaustive background descrip
tion of the model architecture and refer readers to
 Vaswani et al. (2017) as well as excellent guides
 such as “The Annotated Transformer.”2
 In this work, we denote the number of layers
 (i.e., Transformer blocks) as L, the hidden size as
 H, and the number of self-attention heads as A.3
 We primarily report results on two model sizes:
 BERTBASE (L=12, H=768, A=12, Total Param
eters=110M) and BERTLARGE (L=24, H=1024,
 A=16, Total Parameters=340M).
 BERTBASE was chosen to have the same model
 size as OpenAI GPT for comparison purposes.
 Critically, however, the BERT Transformer uses
 bidirectional self-attention, while the GPT Trans
former uses constrained self-attention where every
 token can only attend to context to its left.4
'''

In [22]:
summary = summarize(input) 

In [23]:
summary

' we introduce a new language representa \ntion model called BERT, which stands for Bidirectional Encoder Representations from Transformers. BERT is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers of the model. \n the pre-trained model can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial taskspecific architecture modifications. \n we propose two approaches to apply pre-training language representations to downstream tasks: feature-based and finetuning based approaches.    in this paper \n, we argue that current techniques restrict the \n power of pre- trained representations by limiting the choice of archi  \ntectures that can be used during pre- training. for example, in the present paper, the authors use a left-to-right architecture, where every token can

Humm Its working....